In [1]:
import random
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Circle
import pandas as pd

import photutils
from photutils import detect_threshold
from astropy.io import fits
from astropy.wcs import WCS
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.modeling.functional_models import Gaussian2D

from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from photutils import detect_sources
import time

import shutil
from math import pi, log, sqrt
from tqdm.auto import tqdm
import fnmatch
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define data paths
DR2_path = "/data/astronomy-big-data/bc96e9620e41b9aba98292d37b5eec24/LoTSS_DR2/"
mosaic_path = "/data/astronomy-big-data/bc96e9620e41b9aba98292d37b5eec24/LoTSS_DR2_mosaic/"
writeable = "/data/astronomy-big-data/bc96e9620e41b9aba98292d37b5eec24/LoTSS_DR2_writable/"

# Calculate radius mosaic

In [3]:
def mosaic(file):
    hdul = fits.open(file)
    # data = hdul[0].data # The intensity data
    conv = hdul[0]._header["BMAJ"] # Should be 1/4 * BMAJ
    
    x = hdul[0]._header['NAXIS1']
    y = hdul[0]._header['NAXIS2']
    
    center = (x/2, y/2)

    # finite_area = np.sum(np.isfinite(data))
    # radius = np.sqrt(finite_area/pi) * conv
    radius = (max(x, y) / 2) * conv * (3/8)
    
    w = WCS(hdul[0].header)
    ra, dec = w.all_pix2world(x/2, y/2, 0, ra_dec_order=True)
    
    return np.array((radius, ra, dec))

In [4]:
fits_files = [mosaic_path + f for f in os.listdir(mosaic_path)]
info = []
for file in fits_files:
    info.append(mosaic(file))
    
info = np.array(info)

# Find overlapping mosaics

In [5]:
overlap_dict = {k.split('/')[-1]: [] for k in fits_files}

for i, f1 in enumerate(fits_files):
    f1 = f1.split('/')[-1]
    for j, f2 in enumerate(fits_files):
        if j > i:
            f2 = f2.split('/')[-1]
            offset = np.array((info[i][1] - info[j][1], info[i][2] - info[j][2]))
            distance = np.sqrt(np.sum(np.power(offset, 2)))
            radius = info[i][0] + info[j][0] 
            if distance <= radius:
                overlap_dict[f1].append(f2)
                overlap_dict[f2].append(f1)

# Pair mosaics

In [6]:
pairs = []
for combs in list(overlap_dict.items()):
    for m in combs[1]:
        p = tuple(sorted([combs[0].split('_')[0], m.split('_')[0]]))
        if p not in pairs:
            pairs.append(p)
print(pairs[:5])

[('P223+50', 'P223+52'), ('P219+50', 'P223+52'), ('P223+52', 'P227+50'), ('P219+52', 'P223+52'), ('P218+55', 'P223+52')]


# Prepare catalogue

In [7]:
header = ["label", "total_pixels", "x_pixels", "y_pixels",
          "integrated_intensity", "brightest_pixel", "brightest_pixel_x", "brightest_pixel_y",
          "brightest_pixel_RA", "brightest_pixel_DEC", "center_of_mass_x", "center_of_mass_y",
          "center_of_mass_RA", "center_of_mass_DEC", "center_of_gaus_fit_x", "center_of_gaus_fit_y",
          "center_of_gaus_fit_RA", "center_of_gaus_fit_DEC", "fit_x_axis", "fit_y_axis", "fit_theta",
          "deconv_x", "deconv_y", "integrated_intensity_fit", "ratio_residual"
         ]
catalogue = pd.read_csv(writeable + "catalogue_v6.0.csv", sep=",", names=header)

# Filter nans
rows = len(catalogue)
nan_rows = catalogue[catalogue.isnull().any(axis=1)]
ratio_nan = len(nan_rows) / rows
print('{0:.2f}% nan rows'.format(ratio_nan * 100))

# Show new catalogue
catalogue = catalogue[~catalogue.index.isin(nan_rows.index)].reindex()
catalogue['overlap'] = 0

# Add mosaic
catalogue['mosaic'], catalogue['object'] = catalogue['label'].str.split('_', 1).str
catalogue['object'] = catalogue['object'].astype(int)
catalogue.head()

0.15% nan rows


,label,total_pixels,x_pixels,y_pixels,integrated_intensity,brightest_pixel,brightest_pixel_x,brightest_pixel_y,brightest_pixel_RA,brightest_pixel_DEC,...,fit_x_axis,fit_y_axis,fit_theta,deconv_x,deconv_y,integrated_intensity_fit,ratio_residual,overlap,mosaic,object
0,P244+48_0,28,7,5,0.658204,0.000776,5389,139,243.974042,45.788265,...,3.229097,1.546218,-0.116900,6.466833,0.000000,0.666203,0.116075,0,P244+48,0
1,P244+48_1,16,5,5,0.362765,0.000715,5608,164,243.843057,45.797968,...,1.368726,1.592619,0.174580,0.000000,0.000000,0.373679,0.171475,0,P244+48,1
2,P244+48_2,252,20,18,89.688294,0.070113,5618,178,243.837009,45.803768,...,2.394096,1.987787,0.599322,3.972817,2.431183,83.919963,0.093798,0,P244+48,2
3,P244+48_3,16,7,3,0.325570,0.000646,5680,177,243.799958,45.803118,...,2.325893,1.161440,-0.003892,3.741414,0.000000,0.330477,0.134046,0,P244+48,3
4,P244+48_4,35,11,7,0.871635,0.000914,5485,186,243.916467,45.807561,...,3.308703,1.257190,0.423071,6.686249,0.000000,0.866827,0.260646,0,P244+48,4


# Make mosaic dict

In [8]:
mosaic_dict = {k.split('_')[0]: i for i, k in enumerate(overlap_dict.keys()) }
mosaic_dict_rev = {i: k.split('_')[0] for i, k in enumerate(overlap_dict.keys()) }
mosaic_dict

{'P223+52': 0,
 'P174+57': 1,
 'P4Hetdex16': 2,
 'P022+34': 3,
 'P18Hetdex03': 4,
 'P205+42': 5,
 'P236+48': 6,
 'P245+55': 7,
 'P10Hetdex': 8,
 'P32Hetdex08': 9,
 'P223+50': 10,
 'P233+60': 11,
 'P121+32': 12,
 'P178+55': 13,
 'P235+50': 14,
 'P12Hetdex11': 15,
 'P228+60': 16,
 'P15Hetdex13': 17,
 'P143+52': 18,
 'P202+42': 19,
 'P219+50': 20,
 'P181+60': 21,
 'P145+57': 22,
 'P217+47': 23,
 'P141+45': 24,
 'P227+50': 25,
 'P226+42': 26,
 'P156+42': 27,
 'P155+60': 28,
 'P163+45': 29,
 'P207+45': 30,
 'P176+60': 31,
 'P238+60': 32,
 'P229+45': 33,
 'P11Hetdex12': 34,
 'P202+60': 35,
 'P35Hetdex10': 36,
 'P236+53': 37,
 'P155+52': 38,
 'P027+31': 39,
 'P345+33': 40,
 'P000+38': 41,
 'P030+39': 42,
 'P145+45': 43,
 'P215+50': 44,
 'P222+60': 45,
 'P5Hetdex': 46,
 'P181+42': 47,
 'P013+26': 48,
 'P34Hetdex06': 49,
 'P151+55': 50,
 'P229+48': 51,
 'P36Hetdex10': 52,
 'P163+42': 53,
 'P017+39': 54,
 'P146+42': 55,
 'P191+55': 56,
 'P193+57': 57,
 'P035+34': 58,
 'P209+42': 59,
 'P8Hetdex':

In [9]:
catalogue["mosaic_id"] = catalogue["mosaic"].apply(lambda x: mosaic_dict[x])
catalogue.loc[:, ["mosaic", "mosaic_id"]]

,mosaic,mosaic_id
0,P244+48,112
1,P244+48,112
2,P244+48,112
3,P244+48,112
4,P244+48,112
...,...,...
2291390,P146+42,55
2291391,P146+42,55
2291392,P146+42,55
2291393,P146+42,55


# Finding overlapping sources

In [10]:
def detectOverlap(data, labels): 
    obj_rows = []
    eps = 0.0016 # Define distance between brightest pixel
    i = 0
    
    while i < len(data)-1:
        # Same mosaic
        if labels[i][0] == labels[i+1][0]:
            i += 1
            continue
        
        if sqrt(np.sum(np.power(data[i] - data[i+1], 2))) <= eps:
            # Add to new rows
            obj_rows.append(np.array(sorted([labels[i], labels[i+1]], key=lambda x: x[0])).astype(int))            
            # Only 2 sources can overlap
            i += 2
        else:
            i += 1
    
    return np.array(obj_rows)

In [11]:
catalogue.sort_values(
    by=["brightest_pixel_RA", "brightest_pixel_DEC"]
).loc[:, ["brightest_pixel_RA", "brightest_pixel_DEC", "mosaic_id", "object", "total_pixels"]].head()

,brightest_pixel_RA,brightest_pixel_DEC,mosaic_id,object,total_pixels
2145219,0.000432,38.996926,41,3985,20
2145211,0.001013,38.993597,41,3977,32
2142110,0.001123,37.227152,41,876,182
2144645,0.001379,38.684410,41,3411,57
2145953,0.001981,39.440752,41,4719,110


In [12]:
select = lambda s, k, v: np.array_split(np.array(
    s[(s["mosaic"] == k) | (s["mosaic"] == v)]
    .sort_values(by=["brightest_pixel_RA", "brightest_pixel_DEC"])
    .loc[:, ["brightest_pixel_RA", "brightest_pixel_DEC", "mosaic_id", "object"]]
), 2, axis=1)


In [13]:
overlap_rows = []
with tqdm(total=len(pairs)) as pbar:
    for pair in pairs:
        data, label = select(catalogue, pair[0], pair[1])
        for p in detectOverlap(data, label.astype(int)):
            overlap_rows.append(p)
        pbar.update(1)
        
overlap_rows[:5]

[array([[   0, 1424],
        [  10, 9671]]),
 array([[   0, 1383],
        [  10, 9613]]),
 array([[   0, 1548],
        [  10, 9813]]),
 array([[   0, 1575],
        [  10, 9834]]),
 array([[   0, 1549],
        [  10, 9816]])]

In [14]:
overlap_rows = np.array(sorted(overlap_rows, key=lambda x: x[0][0]))
overlap_rows[:5]

array([[[   0, 1424],
        [  10, 9671]],

       [[   0, 1383],
        [  10, 9613]],

       [[   0, 1548],
        [  10, 9813]],

       [[   0, 1575],
        [  10, 9834]],

       [[   0, 1549],
        [  10, 9816]]])

# Group pairs

In [15]:
groups = {}
with tqdm(len(overlap_rows)) as pbar:
    for mat in overlap_rows:
        key = tuple(mat[0])
        val = tuple(mat[1])
        try:
            groups[key].append(val)
        except:
            groups[key] = [val]
        pbar.update(1)

i = 0
for k in groups.keys():
    print(k, groups[k])
    if i == 5:
        break
    i += 1


(0, 1424) [(10, 9671)]
(0, 1383) [(10, 9613)]
(0, 1548) [(10, 9813), (20, 10388)]
(0, 1575) [(10, 9834), (87, 1450)]
(0, 1549) [(10, 9816)]
(0, 1306) [(10, 9513), (20, 10043)]


# Label overlapping sources

In [16]:
np_cat = catalogue.loc[:, ["mosaic_id", "object", "total_pixels"]].values
print(np_cat[:5])

[[112   0  28]
 [112   1  16]
 [112   2 252]
 [112   3  16]
 [112   4  35]]


In [17]:
# lookup = lambda d, arr: int(d[(d["mosaic_id"] == arr[0]) & (d["object"] == arr[1])]["total_pixels"].values)
lookup = lambda d, arr: d[np.where((np_cat[:, 0] == arr[0]) & (np_cat[:, 1] == arr[1] ))][0][-1]

def flatten(L):
    for item in L:
        try:
            yield from flatten(item)
        except TypeError:
            yield item
            
            
def labelOverlap(combs, p_arg):
    overlap_arr = np.ones((len(combs), 3), dtype=int)
    overlap_arr[:, :2] = combs
    overlap_arr[p_arg, 2] += 1
    return overlap_arr

In [18]:
flatten_source_labels = []
with tqdm(total=len(groups.keys())) as pbar:
    for key, val in groups.items():
        val.append(key)
        for row in labelOverlap(val, np.argmax([lookup(np_cat, v) for v in val])):
            flatten_source_labels.append(row)
        pbar.update(1)
        
flatten_source_labels = np.array(flatten_source_labels)
print(flatten_source_labels.shape)
flatten_source_labels[:5]


(821118, 3)


array([[  10, 9671,    2],
       [   0, 1424,    1],
       [  10, 9613,    2],
       [   0, 1383,    1],
       [  10, 9813,    1]])

# Merge and filter catalogue

In [19]:
df_source_labels = pd.DataFrame(flatten_source_labels, columns=["mosaic_id", "object", "overlap"])
df_source_labels.head()

,mosaic_id,object,overlap
0,10,9671,2
1,0,1424,1
2,10,9613,2
3,0,1383,1
4,10,9813,1


In [20]:
unfilt = len(df_source_labels)

df_source_labels = df_source_labels.sort_values(['mosaic_id', 'object', 'overlap']).drop_duplicates(['mosaic_id', 'object'], keep='first')

print("Filtered sources:", len(df_source_labels) - unfilt)

Filtered sources: -59476


In [21]:
new_cat = pd.merge(catalogue, df_source_labels, how='left', on=['mosaic_id', 'object'])
new_cat.head()

,label,total_pixels,x_pixels,y_pixels,integrated_intensity,brightest_pixel,brightest_pixel_x,brightest_pixel_y,brightest_pixel_RA,brightest_pixel_DEC,...,fit_theta,deconv_x,deconv_y,integrated_intensity_fit,ratio_residual,overlap_x,mosaic,object,mosaic_id,overlap_y
0,P244+48_0,28,7,5,0.658204,0.000776,5389,139,243.974042,45.788265,...,-0.116900,6.466833,0.000000,0.666203,0.116075,0,P244+48,0,112,2.0
1,P244+48_1,16,5,5,0.362765,0.000715,5608,164,243.843057,45.797968,...,0.174580,0.000000,0.000000,0.373679,0.171475,0,P244+48,1,112,NaN
2,P244+48_2,252,20,18,89.688294,0.070113,5618,178,243.837009,45.803768,...,0.599322,3.972817,2.431183,83.919963,0.093798,0,P244+48,2,112,NaN
3,P244+48_3,16,7,3,0.325570,0.000646,5680,177,243.799958,45.803118,...,-0.003892,3.741414,0.000000,0.330477,0.134046,0,P244+48,3,112,NaN
4,P244+48_4,35,11,7,0.871635,0.000914,5485,186,243.916467,45.807561,...,0.423071,6.686249,0.000000,0.866827,0.260646,0,P244+48,4,112,2.0


In [22]:
new_cat['overlap'] = np.max(new_cat[['overlap_x', 'overlap_y']], axis=1).astype(int)
new_cat = new_cat.drop(labels=['overlap_x', 'overlap_y'], axis=1)
new_cat.head()

,label,total_pixels,x_pixels,y_pixels,integrated_intensity,brightest_pixel,brightest_pixel_x,brightest_pixel_y,brightest_pixel_RA,brightest_pixel_DEC,...,fit_y_axis,fit_theta,deconv_x,deconv_y,integrated_intensity_fit,ratio_residual,mosaic,object,mosaic_id,overlap
0,P244+48_0,28,7,5,0.658204,0.000776,5389,139,243.974042,45.788265,...,1.546218,-0.116900,6.466833,0.000000,0.666203,0.116075,P244+48,0,112,2
1,P244+48_1,16,5,5,0.362765,0.000715,5608,164,243.843057,45.797968,...,1.592619,0.174580,0.000000,0.000000,0.373679,0.171475,P244+48,1,112,0
2,P244+48_2,252,20,18,89.688294,0.070113,5618,178,243.837009,45.803768,...,1.987787,0.599322,3.972817,2.431183,83.919963,0.093798,P244+48,2,112,0
3,P244+48_3,16,7,3,0.325570,0.000646,5680,177,243.799958,45.803118,...,1.161440,-0.003892,3.741414,0.000000,0.330477,0.134046,P244+48,3,112,0
4,P244+48_4,35,11,7,0.871635,0.000914,5485,186,243.916467,45.807561,...,1.257190,0.423071,6.686249,0.000000,0.866827,0.260646,P244+48,4,112,2


In [23]:
len(new_cat[new_cat["overlap"] > 0])

761642

In [24]:
len(new_cat[new_cat["overlap"] == 1])

398501

In [25]:
len(new_cat[new_cat["overlap"] == 2])

363141

In [26]:
len(new_cat[(new_cat["overlap"] == 0) | (new_cat["overlap"] == 2)])

1889378